# Calculate Angles

Calculate dip angles on all ALHIC1901 datasets

## Setup

In [1]:
# Load packages

import os
os.environ['MKL_SERVICE_FORCE_INTEL'] = '1'

# general
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

# math
from scipy.stats import pearsonr
import math

# my functions/classes
import sys
sys.path.append("../core_scripts/")
from ECMclass import ECM


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Set filepaths

path_to_data = '../../data/ecm/'
path_to_angles = '../../data/angles/'
path_to_figures = '../../../figures/angles/'

In [3]:
# Set parameters

# smoothing window
window = 1

# spacing of line that we interpolate onto
interp_int = 0.0001

# set angles to cycle through
ang = 55
ang_res = 5 #fraction of angle
test_angle = np.linspace(-ang,ang,2*ang*ang_res+1)

## Load Data

In [4]:
# Load metadata
meta = pd.read_csv(path_to_data + 'metadata.csv')
meta.head()

# for now, trim to just the first three rows
#meta = meta.iloc[5:8]

,core,time,section,face,ACorDC,Y_left,Y_right,AC_edgespace,DC_edgespace
0,alhic1901,cmc1,230_4,t,DC,-5.080,206.763,15,10
1,alhic1901,cmc1,230_4,l,DC,70.404,170.096,15,10
2,alhic1901,cmc1,228_4,t,DC,-10.000,192.000,15,10
3,alhic1901,cmc1,228_4,l,AC,63.881,166.250,15,10
4,alhic1901,cmc1,228_4,l,DC,65.891,149.177,15,10


In [5]:
# Load ECM data

data = []
cores = []
sections = []
faces = []
ACorDCs = []
for index, row in tqdm(meta.iterrows(), total=len(meta), desc="Processing data"):
    
    core = row['core']
        
    section = row['section']
    face = row['face']
    ACorDC = row['ACorDC']

    data_item = ECM(core,section,face,ACorDC)
    print("Reading "+core+", section "+section+'-'+face+'-'+ACorDC)
    
    data_item.rem_ends(15)
    data_item.smooth(window)
    data.append(data_item)
    data_item.norm_all()
    
    cores.append(core)
    sections.append(section)
    faces.append(face)
    ACorDCs.append(ACorDC)




Processing data:   0%|          | 0/13 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../../data/metadata.csv'

## Preform Angle Calculations

In [ ]:
angles = []
scores = []
lengths = []
all_scores = []

# loop through each data file
for data_face in data:

    # print summary
    print("Calculating Angle: "+data_face.core+ 
          ", section "+data_face.section+ '-'+data_face.face+
          '-'+data_face.ACorDC)
    
    

    # make empty depth/angle
    score_list = []
    angle=[]
    angle_depth=[]

    # assign local variables
    depth = data_face.depth_s
    meas = data_face.meas_s
    y_list = data_face.y_s
    y_vec = data_face.y_vec

    # make empty array to score all scores
    all_score = np.zeros((len(test_angle),len(y_vec)-1))

    # flip the depth, meas, and y_list vectors
    depth = np.flip(depth)
    meas = np.flip(meas)
    y_list = np.flip(y_list)

    # make empty list of angle and scores
    angle_list = []
    score_list = []
    length_list = []

    # loop through each pair of adjasent tracks
    for i in range(len(y_vec)-1):

        # get two y_values
        y1 = y_vec[i]
        y2 = y_vec[i+1]
        y_space = abs(y2 - y1)

        # pull out data for both tracks
        idx1 = y_list == y1
        idx2 = y_list == y2
        meas1 = meas[idx1]
        meas2 = meas[idx2]
        depth1 = depth[idx1]
        depth2 = depth[idx2]

        run_score = []

        # loop through each angle
        a_cnt = 0
        for a in test_angle:
            
            # caluclate the spacing between tracks
            yspace = y1-y2

            # calculate the change in depth at this slope
            delta_depth = yspace/1000 * np.tan(a * np.pi/180) * 0.5

            # calculate the new depth vectors
            depth1_shifted = depth1 - delta_depth
            depth2_shifted = depth2 + delta_depth

            # Find the overlapping range between the two depth vectors
            min_overlap = max(min(depth1_shifted), min(depth2_shifted))
            max_overlap = min(max(depth1_shifted), max(depth2_shifted))

            # Round to the nearest 1mm (0.001), rounding min up and max down
            min_overlap_rounded = math.ceil(min_overlap * 1000) / 1000
            max_overlap_rounded = math.floor(max_overlap * 1000) / 1000

            # create new depth vector to interpolate onto
            depth_vec = np.arange(min_overlap_rounded, max_overlap_rounded, interp_int)
            length = max(depth_vec) - min(depth_vec)

            # interpolate the two tracks onto the new depth vector
            meas1_interp = np.interp(depth_vec, depth1_shifted, meas1)
            meas2_interp = np.interp(depth_vec, depth2_shifted, meas2)

            # calculate the correlation
            corr, pval = pearsonr(meas1_interp, meas2_interp)

            # caluclate the score
            run_score.append(corr)

            # save score in 2D array
            all_score[a_cnt,i] = corr
            a_cnt += 1

        # find the best score and angle
        best_score = max(run_score)
        best_angle = test_angle[run_score.index(best_score)]

        # append to the lists
        angle_list.append(best_angle)
        score_list.append(best_score)
        

    # Print rough stats
    print("    Mean Angle = "+str(np.mean(angle_list)))
    print("    Mean Angles = "+str(angle_list))
        
    # save the angles, corrs, and lengths
    angles.append(angle_list)
    scores.append(score_list)
    lengths.append(length_list)      
    all_scores.append(all_score)     

In [ ]:
# Save the results

# Set up a df to store the results
face_df = meta.drop(columns=['Y_left', 'Y_right','AC_edgespace','DC_edgespace']).copy()

# save values
face_df['angle'] = angles
face_df['score'] = scores
face_df['length'] = lengths
face_df['all_scores'] = all_scores

# save the dataframe
face_df.head()

## Diagnostic Plot

Shows the correlation strength as a function of angle for each track. Saves plot, and picks out a couple to show here.

In [ ]:
# loop through rows of face_df
for idx, row in face_df.iterrows():

    all_scores = row['all_scores']

    plt.figure(figsize=(10, 6))
    plt.pcolormesh(range(all_scores.shape[1]), test_angle, all_scores, cmap='viridis', shading='auto', norm=plt.Normalize(vmin=0.8, vmax=1))
    plt.colorbar(label='Score')
    plt.title(row['core']+' - '+row['section']+' - '+row['face']+' - '+row['ACorDC'])
    plt.xlabel('Track Pair Index')
    plt.ylabel('Angle (degrees)')
    plt.gca().invert_yaxis()  # Invert y-axis to match the order of test_angle

    # Add black dots at the maximum point in each column
    max_indices = np.argmax(all_scores, axis=0)
    max_scores = all_scores[max_indices, range(all_scores.shape[1])]
    max_angles = test_angle[max_indices]
    mean_angles = np.mean(max_angles)
    std_angles = np.std(max_angles)
    mean_score = np.mean(max_scores)
    std_score = np.std(max_scores)

    mask = (max_scores < mean_score - 2* std_score) | (np.abs(test_angle[max_indices] - mean_angles) > 2*std_angles)

    plt.scatter(range(all_scores.shape[1]), test_angle[max_indices], color='black', s=10, label='Max Points')
    plt.scatter(np.where(mask)[0], test_angle[max_indices[mask]], color='red', s=10, label='High Score Points')

    # Save the figure
    plt.savefig(path_to_figures + 'face_diagnostic/'+ row['core']+'-'+row['section']+'-'+row['face']+'-'+row['ACorDC']+'_angle.png', dpi=300)

    if row['ACorDC'] == 'AC' and row['face'] == 't':
        plt.show()
    else:
        plt.close()

    

## Compute True Angles

In [ ]:
def filter(angles,scores):

    # calc mean/std for masking
    mean_scores = np.mean(scores)
    std_scores = np.std(scores)
    mean_angles = np.mean(angles)
    std_angles = np.std(angles)

       # mask the scores
    mask = (scores < mean_scores - 2* std_scores) | (np.abs(angles - mean_angles) > 2*std_angles)

    # mask the angles and scores
    angles = angles[~mask]
    scores = scores[~mask]

    return angles, scores

In [ ]:
def calc_weighted_mean_face(face_df,face,section):
    # function to calculate the weighted mean angle for a given face and section

    # filter the dataframe
    df = face_df[(face_df['section'] == section) & 
                    (face_df['ACorDC'] == 'AC') & 
                    (face_df['face'] == face)]

    # pull out scores and angles
    scores = np.array(df.score.to_list())[0]
    angles = np.array(df.angle.to_list())[0]

    angles, scores = filter(angles,scores)

    # compute weighted average
    weighted_ave = np.sum(angles * scores)/ np.sum(scores)

    return weighted_ave
    

In [ ]:


def calc_true_dip(face_df,section):
    # function to calculate the true dip for a given section

    # filter the dataframe
    df = face_df[(face_df['section'] == section) & 
                    (face_df['ACorDC'] == 'AC')]

    top_scores = np.array(df[df['face'] == 't'].score.values[0])
    left_scores = np.array(df[df['face'] == 'l'].score.values[0])
    right_scores = np.array(df[df['face'] == 'r'].score.values[0])
    side_scores = np.concatenate((left_scores, right_scores))

    top_angles = np.array(df[df['face'] == 't'].angle.values[0])
    left_angles = np.array(df[df['face'] == 'l'].angle.values[0])
    right_angles = np.array(df[df['face'] == 'r'].angle.values[0]) * -1
    side_angles = np.concatenate((left_angles, right_angles))

    # filter out the bad values
    top_angles, top_scores = filter(top_angles,top_scores)
    side_angles, side_scores = filter(side_angles,side_scores)

    true_angles = []
    true_scores = []
    for top_a,top_s in zip(top_angles,top_scores):
        for side_a,side_s in zip(side_angles,side_scores):
            
            # compute the true dip
            alpha = np.arctan(np.tan(side_a * np.pi/180) / np.tan(top_a * np.pi/180))
            true_angles.append(np.arctan(np.tan(top_a * np.pi/180)/np.cos(alpha)))
            true_scores.append(top_s * side_s) 

    return np.array(true_angles), np.array(true_scores)

a,s = calc_true_dip(face_df,'228_4')

In [ ]:
angles_df = pd.DataFrame()

wt_t = 0
wt_l = 0
wt_r = 0

# loop through each section
for section in ['228_4','230_4']:

    for face, weighted_ave in zip(['t','l','r'],[wt_t,wt_l,wt_r]):

        # compute weighted mean on each face
        wt_t = calc_weighted_mean_face(face_df,'t',section)
        wt_l = calc_weighted_mean_face(face_df,'l',section)
        wt_r = calc_weighted_mean_face(face_df,'r',section)  
        wt_side = (wt_l - wt_r)/2

    a,s = calc_true_dip(face_df,section)

    true_dip = np.sum(a * s) /np.sum(s)

    plt.figure(figsize=(10, 6))
    plt.scatter(a*180/np.pi, s, label='True Dip Scores')
    plt.axvline(x=true_dip*180/np.pi, color='red', linestyle='--', label='True Dip = '+str(round(true_dip*180/np.pi,2)))
    plt.xlabel('True Dip Angle (degrees)')
    plt.ylabel('True Dip Score')
    plt.title('True Dip for section ' + section)
    plt.legend()
    plt.savefig(path_to_figures + section+'_true_dip.png', dpi=300)
    plt.show()

    # save the results
    angles_df = pd.concat([angles_df, pd.DataFrame({
        'core': ['alhic1901'],
        'section': [section],
        'ACorDC': [ACorDC],
        'face': [face],
        'top_angle': [wt_t],
        'left_angle': [wt_l],
        'right_angle': [wt_r],
        'side_angle': [wt_side],
        'true_dip': [true_dip*180/np.pi],
        'true_dip_angles': [a*180/np.pi],
        'true_dip_scores': [s],
    })], ignore_index=True)
            
# save the angles_df
angles_df.to_csv(path_to_angles + 'angles.csv', index=False)
angles_df.head()